$I -\textbf{Fetching \& processing the data}$

$\quad\quad\quad \quad 0-\textbf{Dependencies}$

In [1]:
from Bio import Entrez,SeqIO 
from Bio.SeqFeature import SeqFeature, FeatureLocation 
import requests  
import pandas as pd  
import numpy as np     
import requests   
from bs4 import BeautifulSoup  
from sklearn.model_selection import train_test_split  
from transformers import AutoModelForSequenceClassification 
import torch 

In [2]:
%load_ext cudf.pandas

In [3]:
torch.cuda.is_available()

True

$\quad\quad\quad\quad 1-\textbf{Fetching}$

Getting the Associations from the GWAS Catalog -- $\textbf{Cancer}$

In [4]:
df=pd.read_csv("/home/kaizokuo/Documents/DATA/cancer_associations.tsv",sep="\t")

Cleaning the data 

In [5]:
# As an array 
ref_mutA =np.array([[x[-1], y.split(':')[0], (y.split(':')[1]).split(',')[0],z.split(',')[0]] for x, y,z in zip(df["riskAllele"], df["locations"],df['mappedGenes']) if x[-1] != '?' and y!='-' ])
# As a DataFrame
ref_mutD =pd.DataFrame(ref_mutA,columns=['riskAllele','Chromosome','MutAll_Pos','MappedGene'])  

Get the location of the gene 

In [6]:
# Function that extracts the gene's location given it's name 
def get_gene_location(gene_name):
    server = "https://rest.ensembl.org"
    ext = f"/lookup/symbol/human/{gene_name}?content-type=application/json"
    response = requests.get(server + ext, headers={"Content-Type": "application/json"})
    if not response.ok:
        return '-'
    
    data = response.json()
    start = data['start']
    end = data['end']
    return start, end

In [7]:
#Gloc=ref_mutD["MappedGene"].apply(get_gene_location)

In [8]:
#Gloctxt=Gloc.to_string(index=False,max_rows=None)

In [9]:
#f = open("Gene_location.txt", 'w')
#f.write(Gloctxt)
#f.close()

In [10]:
file=open("/home/kaizokuo/Documents/Jupyter/Gene_location.txt","r")

In [11]:
Gloc=file.readlines()

In [12]:
Gloc=[line.replace("(","").replace(")","").replace(" ","").replace("\n","") for line in Gloc]  

In [13]:
ref_mutD 

,riskAllele,Chromosome,MutAll_Pos,MappedGene
0,A,1,1351675,MXRA8
1,T,1,155212373,MTX1
2,T,3,27285723,NEK10
3,A,4,105140377,TET2
4,A,5,1282299,TERT
...,...,...,...,...
11516,C,6,53312880,ELOVL5
11517,C,18,53580709,LINC01919
11518,C,5,87315172,RASA1
11519,C,3,175534948,NAALADL2


In [14]:
ref_mutA =np.array([[str(x),str(y),str(z),str(a)] for x,y,z,a in zip(ref_mutD["riskAllele"], ref_mutD["Chromosome"],ref_mutD["MutAll_Pos"],Gloc) if a!='-'])
ref_mutD =pd.DataFrame(ref_mutA,columns=['riskAllele','Chromosome','MutAllLoc','MapGenLoc'])  

In [15]:
ref_mutD

,riskAllele,Chromosome,MutAllLoc,MapGenLoc
0,A,1,1351675,"1352689,1361777"
1,T,1,155212373,"155208695,155213824"
2,T,3,27285723,"27106484,27369460"
3,A,4,105140377,"105145875,105279816"
4,A,5,1282299,"1253147,1295068"
...,...,...,...,...
9704,C,6,53312880,"53267398,53349179"
9705,C,18,53580709,"53568439,53629990"
9706,C,5,87315172,"87267883,87391931"
9707,C,3,175534948,"174438573,175810548"


For the Reference Gene (this didn't work)

In [16]:
"""import cudf
ref_mutD_cdf=cudf.DataFrame.from_pandas(ref_mutD)"""

'import cudf\nref_mutD_cdf=cudf.DataFrame.from_pandas(ref_mutD)'

Error : this won't work because cudf doesn't accept strings. 

In [17]:
def fetch_gene_sequence_ensembl_cudf(chr, maloc, mgloc,out_col):
    for i, (c1,c2,c3) in enumerate(zip(chr,maloc,mgloc)):
        print(i)
        chromosome=c1
        h=c3.split(',')
        start=int(h[0])
        end=int(h[1])
        al=int(c2) 
        if start>al:
            return '-'
        # reducing the gene size for the tokenizer + faster for computer 
        start=max(al-250,start)
        end=min(al+250,end)
        assembly="GRCh38"
        server = "https://rest.ensembl.org"
        ext = f"/sequence/region/human/{chromosome}:{start}..{end}:{1}?coord_system_version={assembly}"
        
        headers = { "Content-Type" : "text/plain"}
        response = requests.get(server+ext, headers=headers)
        if not response.ok:
            out_col[i]='-'
        
        else :
            out_col[i]=response.text


            



We can preprocess the data to make the input acceptable

In [18]:
"""start=np.zeros(len(ref_mutD))
end=np.zeros(len(ref_mutD))
for i, (c1,c2,c3) in enumerate(zip(ref_mutD["Chromosome"],ref_mutD["MutAllLoc"],ref_mutD["MapGenLoc"])):
    print(i)
    chromosome=c1
    h=c3.split(',')
    st=int(h[0])
    ed=int(h[1])
    al=int(c2) 
    if st>al:
        continue
    # reducing the gene size for the tokenizer + faster for computer 
    start[i]=max(al-250,st)
    end[i]=min(al+250,ed)"""

'start=np.zeros(len(ref_mutD))\nend=np.zeros(len(ref_mutD))\nfor i, (c1,c2,c3) in enumerate(zip(ref_mutD["Chromosome"],ref_mutD["MutAllLoc"],ref_mutD["MapGenLoc"])):\n    print(i)\n    chromosome=c1\n    h=c3.split(\',\')\n    st=int(h[0])\n    ed=int(h[1])\n    al=int(c2) \n    if st>al:\n        continue\n    # reducing the gene size for the tokenizer + faster for computer \n    start[i]=max(al-250,st)\n    end[i]=min(al+250,ed)'

Numerize Chromosome column

In [19]:
"""for i in range(len(ref_mutD)):
    if ref_mutD["Chromosome"][i]=="X":
        ref_mutD["Chromosome"][i]=24
    elif  ref_mutD["Chromosome"][i]=="Y":
        ref_mutD["Chromosome"][i]=23
    else : 
        ref_mutD["Chromosome"][i]=int(ref_mutD["Chromosome"][i])
ref_mutD["Chromosome"] = ref_mutD["Chromosome"].apply(lambda x: int(x))"""

'for i in range(len(ref_mutD)):\n    if ref_mutD["Chromosome"][i]=="X":\n        ref_mutD["Chromosome"][i]=24\n    elif  ref_mutD["Chromosome"][i]=="Y":\n        ref_mutD["Chromosome"][i]=23\n    else : \n        ref_mutD["Chromosome"][i]=int(ref_mutD["Chromosome"][i])\nref_mutD["Chromosome"] = ref_mutD["Chromosome"].apply(lambda x: int(x))'

In [20]:
"""type(ref_mutD["Chromosome"])"""

'type(ref_mutD["Chromosome"])'

In [21]:
"""start_end=cudf.DataFrame({
    "start":start,
    "end":end,
    "Chromosome":ref_mutD["Chromosome"]
})
print(start_end)"""

'start_end=cudf.DataFrame({\n    "start":start,\n    "end":end,\n    "Chromosome":ref_mutD["Chromosome"]\n})\nprint(start_end)'

New UDF for CudF

In [22]:
"""hash={'A':1,'C':2,'T':3,'G':4,'i':5,'D':6}


def fetch_gene_sequence_ensembl_cudf(start,end,Chromosome,RefGen):
    for i, (st,ed,chr) in enumerate(zip(start,end,Chromosome)):
        print(i)
        if st==ed==0:
            RefGen[i]="-"
        server = "https://rest.ensembl.org"
        ext = f"/sequence/region/human/{chr}:{st}..{ed}:{1}?coord_system_version=GRCh38"
        
        headers = { "Content-Type" : "text/plain"}
        response = requests.get(server+ext, headers=headers)
        if not response.ok:
            RefGen[i]=-1
        
        else :
            dna=response.text
            dna=list(dna)
            for i in range(len(dna)):
                if dna[i]=='A':
                    dna[i]=1
                elif dna[i]=='C':
                    dna[i]=2
                elif dna[i]=='T':
                    dna[i]=3
                elif dna[i]=='G':
                    dna[i]=4
                elif dna[i]=='I':
                    dna[i]=5
                elif dna[i]=='D':
                    dna[i]=6
            RefGen[i]=dna"""

'hash={\'A\':1,\'C\':2,\'T\':3,\'G\':4,\'i\':5,\'D\':6}\n\n\ndef fetch_gene_sequence_ensembl_cudf(start,end,Chromosome,RefGen):\n    for i, (st,ed,chr) in enumerate(zip(start,end,Chromosome)):\n        print(i)\n        if st==ed==0:\n            RefGen[i]="-"\n        server = "https://rest.ensembl.org"\n        ext = f"/sequence/region/human/{chr}:{st}..{ed}:{1}?coord_system_version=GRCh38"\n        \n        headers = { "Content-Type" : "text/plain"}\n        response = requests.get(server+ext, headers=headers)\n        if not response.ok:\n            RefGen[i]=-1\n        \n        else :\n            dna=response.text\n            dna=list(dna)\n            for i in range(len(dna)):\n                if dna[i]==\'A\':\n                    dna[i]=1\n                elif dna[i]==\'C\':\n                    dna[i]=2\n                elif dna[i]==\'T\':\n                    dna[i]=3\n                elif dna[i]==\'G\':\n                    dna[i]=4\n                elif dna[i]==\'I\':

In [23]:
"""refGen=start_end.apply_rows(fetch_gene_sequence_ensembl_cudf,
                                     incols=["start","end","Chromosome"],
                                     outcols=dict(RefGen=int),
                                     kwargs={})"""

'refGen=start_end.apply_rows(fetch_gene_sequence_ensembl_cudf,\n                                     incols=["start","end","Chromosome"],\n                                     outcols=dict(RefGen=int),\n                                     kwargs={})'

In [24]:
ref_mutD["Chromosome"].unique()

array(['1', '3', '4', '5', '7', '9', '10', '11', '12', '13', '14', '17',
       '19', '20', '6', '18', '16', '22', '8', '2', '15', '21', 'X'],
      dtype=object)

Fetching DNA Sequence :

In [25]:
def fetch_gene_sequence_ensembl(X):
    print(X.name)
    chromosome=X.iloc[1]
    h=X.iloc[3].split(',')
    start=int(h[0])
    end=int(h[1])
    al=int(X.iloc[2]) 
    if start>al or al>end: 
        return '-'
    # reducing the gene size for the tokenizer + faster for computer 
    #start=max(al-250,start) 
    #end=min(al+250,end)
    assembly="GRCh38"
    server = "https://rest.ensembl.org"
    ext = f"/sequence/region/human/{chromosome}:{start}..{end}:{1}?coord_system_version={assembly}"
    
    headers = { "Content-Type" : "text/plain"}
    response = requests.get(server+ext, headers=headers)
    if not response.ok:
        return '-'
    return response.text

Saving the Reference Gene

In [26]:
#ref_mutD["RefGene"]=pd.DataFrame(ref_mutD.apply(fetch_gene_sequence_ensembl,axis=1))

In [27]:
#with open("Reference_gene.txt", "w") as file:
    # Iterate through each sequence in the list
#    for sequence in ref_mutD["RefGene"]:
        # Write the sequence to the file, followed by a newline character
#        file.write(sequence + "\n")

In [28]:
f=open("Reference_gene.txt")
RefGene=f.readlines()

In [29]:
ref_mutD["RefGene"]=RefGene

In [30]:
ref_mutD

,riskAllele,Chromosome,MutAllLoc,MapGenLoc,RefGene
0,A,1,1351675,"1352689,1361777",-\n
1,T,1,155212373,"155208695,155213824",ACAGAAGTACAATGCTGATTATGATCTGTCAGCTCGGCAAGGGGCA...
2,T,3,27285723,"27106484,27369460",TGAAAATAACACACATGATAGAAATTTCTGTTATGTCTTTCAAAAG...
3,A,4,105140377,"105145875,105279816",-\n
4,A,5,1282299,"1253147,1295068",CTGCACTCCAGCCTGGGCAACAGAGTGAGACTCCATCTCAAAAAAG...
...,...,...,...,...,...
9704,C,6,53312880,"53267398,53349179",TCCTGTACAATGTTACCACTGGGGGAAACAGGGTAGAGAGTACTTG...
9705,C,18,53580709,"53568439,53629990",CACCATCTTCTTGTTTTTTTCTTGCAGAAGAAATGTCAAGTTATAC...
9706,C,5,87315172,"87267883,87391931",GATAAGTGACACCCCCATCTTTAAGTTTGTTTCAGAGATCCTAAAT...
9707,C,3,175534948,"174438573,175810548",GGTATTTAAATGATAAAGGAGGCCTATTGACTCAGTAGTTTAGGAC...


In [31]:
ref_mutA =np.array([[str(x),str(y),str(z),str(a),str(b)] for x,y,z,a,b in zip(ref_mutD["riskAllele"], ref_mutD["Chromosome"],ref_mutD["MutAllLoc"],ref_mutD["MapGenLoc"],ref_mutD["RefGene"]) if a!='-' and b!='-\n'])
ref_mutD =pd.DataFrame(ref_mutA,columns=['riskAllele','Chromosome','MutAllLoc','MapGenLoc','RefGene']) 

In [32]:
ref_mutD

,riskAllele,Chromosome,MutAllLoc,MapGenLoc,RefGene
0,T,1,155212373,"155208695,155213824",ACAGAAGTACAATGCTGATTATGATCTGTCAGCTCGGCAAGGGGCA...
1,T,3,27285723,"27106484,27369460",TGAAAATAACACACATGATAGAAATTTCTGTTATGTCTTTCAAAAG...
2,A,5,1282299,"1253147,1295068",CTGCACTCCAGCCTGGGCAACAGAGTGAGACTCCATCTCAAAAAAG...
3,G,7,18935100,"18086949,19002416",CATGGCGACGTTATTTGTAACGACAAAAAAACTGGAATTTGGTGAT...
4,A,7,79974067,"79768028,80226181",GGATCATATGTTAAGTGCATGTTTAGTTTTGTAAGAAACTGCCAAA...
...,...,...,...,...,...
6396,C,6,53312880,"53267398,53349179",TCCTGTACAATGTTACCACTGGGGGAAACAGGGTAGAGAGTACTTG...
6397,C,18,53580709,"53568439,53629990",CACCATCTTCTTGTTTTTTTCTTGCAGAAGAAATGTCAAGTTATAC...
6398,C,5,87315172,"87267883,87391931",GATAAGTGACACCCCCATCTTTAAGTTTGTTTCAGAGATCCTAAAT...
6399,C,3,175534948,"174438573,175810548",GGTATTTAAATGATAAAGGAGGCCTATTGACTCAGTAGTTTAGGAC...


For the Mutated Gene

In [33]:
def Mutate(X):
    MutGene=X["RefGene"]
    a=int(X["MapGenLoc"].split(',')[0])
    b=int(X["MutAllLoc"])
    if b-250<=a:
        pos_on_gene=b-a
    else : 
        pos_on_gene=250
    MutGene=list(MutGene)
    MutGene[pos_on_gene]=X["riskAllele"]
    MutGene=''.join(MutGene)
    return MutGene 

In [34]:
ref_mutD["MutGene"]=pd.DataFrame(ref_mutD.apply(Mutate,axis=1))

In [35]:
ref_mutD

,riskAllele,Chromosome,MutAllLoc,MapGenLoc,RefGene,MutGene
0,T,1,155212373,"155208695,155213824",ACAGAAGTACAATGCTGATTATGATCTGTCAGCTCGGCAAGGGGCA...,ACAGAAGTACAATGCTGATTATGATCTGTCAGCTCGGCAAGGGGCA...
1,T,3,27285723,"27106484,27369460",TGAAAATAACACACATGATAGAAATTTCTGTTATGTCTTTCAAAAG...,TGAAAATAACACACATGATAGAAATTTCTGTTATGTCTTTCAAAAG...
2,A,5,1282299,"1253147,1295068",CTGCACTCCAGCCTGGGCAACAGAGTGAGACTCCATCTCAAAAAAG...,CTGCACTCCAGCCTGGGCAACAGAGTGAGACTCCATCTCAAAAAAG...
3,G,7,18935100,"18086949,19002416",CATGGCGACGTTATTTGTAACGACAAAAAAACTGGAATTTGGTGAT...,CATGGCGACGTTATTTGTAACGACAAAAAAACTGGAATTTGGTGAT...
4,A,7,79974067,"79768028,80226181",GGATCATATGTTAAGTGCATGTTTAGTTTTGTAAGAAACTGCCAAA...,GGATCATATGTTAAGTGCATGTTTAGTTTTGTAAGAAACTGCCAAA...
...,...,...,...,...,...,...
6396,C,6,53312880,"53267398,53349179",TCCTGTACAATGTTACCACTGGGGGAAACAGGGTAGAGAGTACTTG...,TCCTGTACAATGTTACCACTGGGGGAAACAGGGTAGAGAGTACTTG...
6397,C,18,53580709,"53568439,53629990",CACCATCTTCTTGTTTTTTTCTTGCAGAAGAAATGTCAAGTTATAC...,CACCATCTTCTTGTTTTTTTCTTGCAGAAGAAATGTCAAGTTATAC...
6398,C,5,87315172,"87267883,87391931",GATAAGTGACACCCCCATCTTTAAGTTTGTTTCAGAGATCCTAAAT...,GATAAGTGACACCCCCATCTTTAAGTTTGTTTCAGAGATCCTAAAT...
6399,C,3,175534948,"174438573,175810548",GGTATTTAAATGATAAAGGAGGCCTATTGACTCAGTAGTTTAGGAC...,GGTATTTAAATGATAAAGGAGGCCTATTGACTCAGTAGTTTAGGAC...


Final Input 

In [36]:
input=[[k,1] for k in ref_mutD["MutGene"]]            
input.extend([[k,0] for k in ref_mutD["RefGene"]])    

In [37]:
input=pd.DataFrame(input,columns=["DnaStrand","labels"])

In [38]:
input

,DnaStrand,labels
0,ACAGAAGTACAATGCTGATTATGATCTGTCAGCTCGGCAAGGGGCA...,1
1,TGAAAATAACACACATGATAGAAATTTCTGTTATGTCTTTCAAAAG...,1
2,CTGCACTCCAGCCTGGGCAACAGAGTGAGACTCCATCTCAAAAAAG...,1
3,CATGGCGACGTTATTTGTAACGACAAAAAAACTGGAATTTGGTGAT...,1
4,GGATCATATGTTAAGTGCATGTTTAGTTTTGTAAGAAACTGCCAAA...,1
...,...,...
12797,TCCTGTACAATGTTACCACTGGGGGAAACAGGGTAGAGAGTACTTG...,0
12798,CACCATCTTCTTGTTTTTTTCTTGCAGAAGAAATGTCAAGTTATAC...,0
12799,GATAAGTGACACCCCCATCTTTAAGTTTGTTTCAGAGATCCTAAAT...,0
12800,GGTATTTAAATGATAAAGGAGGCCTATTGACTCAGTAGTTTAGGAC...,0


$II -\textbf{Fine Tuning DNABERT-2}$

$\quad\quad\quad\quad 0-\textbf{Dependencies}$

In [39]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from datasets import Dataset
from sklearn.metrics import f1_score
from datasets import load_dataset, DatasetDict
from transformers import Trainer

$\quad\quad\quad\quad 1-\textbf{Fine Tuning}$

In [40]:
input_raw=Dataset.from_pandas(input)

In [41]:
input_raw=input_raw.train_test_split(test_size=0.2)
inter=input_raw["train"].train_test_split(test_size=0.1)
input_raw=DatasetDict({
    'train':inter["train"],
    'validation':inter["test"],
    'test':input_raw["test"]
})

In [42]:
input_raw["train"][1800]

{'DnaStrand': 'ACTCCAGCCTGGGCGACAGGGAGACTCCGTCTCAAAAAAAAAAAAAAAAAAAAAGGCTGCTCCCCCTCACACCCCCCAACGTCCAGCAGCGGCAACCTGCTCGGGTACCCTTCCACGCTGTGGAAGCTTTGTTCTTTGCTCTTCACAATAAATCTTGCTGCTGCTCACTCTTTGGGTCCACGCCACCTGTAAGAGCTGTAACACTCACCGCGAAGGTTCATGGCTTCATTCTTGACGTCAGCGAGACCACGAGCCCACCTGAAGGAACCAACTCCTGACACAGAAGCACAGGCCATCTCCCTAGGCCCAGCTGCCTTTTGAGGGTGCTCCCAGGTGTCCACAGTTTAAGATCACCGAGTATCTGCTCTAGTTAGAAGGATGCATCAAGGAATGGGAAGCATTGGCTGAGGGGAACCACTAAGAAACACCTGCCTGGGCTGGCCTTTCCCCTGGGTGTGTCCACAGACAGCTTTTGCCCAAGAATTTCAGTATAGGTGAGAA\n',
 'labels': 0}

In [43]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [44]:
torch.cuda.is_available()

True

In [45]:
from transformers.models.bert.configuration_bert import BertConfig
from transformers import AutoModel,AutoTokenizer,BertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
checkpoint = BertConfig.from_pretrained("zhihan1996/DNABERT-2-117M")

In [46]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels=2
model = (BertForSequenceClassification.from_pretrained("zhihan1996/DNABERT-2-117M", num_labels=num_labels).to(device))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at zhihan1996/DNABERT-2-117M and are newly initialized: ['bert.embeddings.position_embeddings.weight', 'bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.0.intermediate.dense.weight', 'bert.encoder.layer.0.output.LayerNorm.bias', 'bert.encoder.layer.0.output.LayerNorm.weight', 'bert.encoder.layer.0.output.dense.bias', 'bert.encoder.layer.0.output.dense.weight', 'bert.encoder.layer.1.attention.self.key.bias', 'bert.encoder.layer.1.attention.self.key.weight', 'bert.encoder.layer.1.attention.self.query.bias', 'bert.encoder.layer.1.attention.self.query.weight', 'bert.encoder.layer.1.at

In [47]:
def tokenize(batch):
    return tokenizer(batch["DnaStrand"], padding=True, truncation=True)

In [48]:
input_encoded = input_raw.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/9216 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1025 [00:00<?, ? examples/s]

Map:   0%|          | 0/2561 [00:00<?, ? examples/s]

In [49]:
from huggingface_hub import notebook_login
notebook_login()

In [50]:
from transformers import Trainer, TrainingArguments
batch_size = 16
logging_steps = len(input_encoded["train"]) // batch_size
model_name = "zhihan1996/DNABERT-2-117M-finetuned-cancer"
training_args = TrainingArguments(output_dir=model_name,
                                num_train_epochs=2,
                                learning_rate=2e-5,
                                per_device_train_batch_size=batch_size,
                                per_device_eval_batch_size=batch_size,
                                weight_decay=0.01,
                                eval_strategy="epoch",
                                disable_tqdm=False,
                                logging_steps=logging_steps,
                                push_to_hub=False,
                                log_level="error")

In [51]:
"""trainer = Trainer(model=model, args=training_args,
                compute_metrics=compute_metrics,
                train_dataset=input_encoded["train"],
                eval_dataset=input_encoded["validation"],
                tokenizer=tokenizer)"""

'trainer = Trainer(model=model, args=training_args,\n                compute_metrics=compute_metrics,\n                train_dataset=input_encoded["train"],\n                eval_dataset=input_encoded["validation"],\n                tokenizer=tokenizer)'

In [52]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [53]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = torch.nn.CrossEntropyLoss()(logits, labels)
        return (loss, outputs) if return_outputs else loss
trainer = CustomTrainer(model=model, args=training_args,
                compute_metrics=compute_metrics,
                train_dataset=input_encoded["train"],
                eval_dataset=input_encoded["validation"],
                tokenizer=tokenizer)

In [54]:
torch.cuda.is_available()

True

In [55]:
trainer.train()

  0%|          | 0/1152 [00:00<?, ?it/s]

{'loss': 0.7073, 'grad_norm': 2.343545913696289, 'learning_rate': 1e-05, 'epoch': 1.0}


  0%|          | 0/65 [00:00<?, ?it/s]

{'eval_loss': 0.6929789781570435, 'eval_accuracy': 0.5092682926829268, 'eval_f1': 0.3436820281583553, 'eval_runtime': 16.8204, 'eval_samples_per_second': 60.938, 'eval_steps_per_second': 3.864, 'epoch': 1.0}
{'loss': 0.6998, 'grad_norm': 1.8301968574523926, 'learning_rate': 0.0, 'epoch': 2.0}


  0%|          | 0/65 [00:00<?, ?it/s]

{'eval_loss': 0.6932951807975769, 'eval_accuracy': 0.5092682926829268, 'eval_f1': 0.3436820281583553, 'eval_runtime': 16.7509, 'eval_samples_per_second': 61.191, 'eval_steps_per_second': 3.88, 'epoch': 2.0}
{'train_runtime': 859.0679, 'train_samples_per_second': 21.456, 'train_steps_per_second': 1.341, 'train_loss': 0.7035545508066813, 'epoch': 2.0}


TrainOutput(global_step=1152, training_loss=0.7035545508066813, metrics={'train_runtime': 859.0679, 'train_samples_per_second': 21.456, 'train_steps_per_second': 1.341, 'total_flos': 1098751767183360.0, 'train_loss': 0.7035545508066813, 'epoch': 2.0})

In [56]:
trainer.evaluate()

  0%|          | 0/65 [00:00<?, ?it/s]

{'eval_loss': 0.6932951807975769,
 'eval_accuracy': 0.5092682926829268,
 'eval_f1': 0.3436820281583553,
 'eval_runtime': 16.7415,
 'eval_samples_per_second': 61.225,
 'eval_steps_per_second': 3.883,
 'epoch': 2.0}

$\quad\quad\quad\quad 2-\textbf{Fine Tuning With Full sequence}$

In [57]:
#ref_mutD["RefGeneLong"]=pd.DataFrame(ref_mutD.apply(fetch_gene_sequence_ensembl,axis=1))

In [58]:
# Iterate through each sequence in the list
#    for sequence in ref_mutD["RefGeneLong"]:
        # Write the sequence to the file, followed by a newline character
#        file.write(sequence + "\n") 

In [59]:
fl=open("Reference_gene_Long.txt")
RefGeneLong=fl.readlines()
ref_mutD["RefGeneLong"]=RefGeneLong

In [68]:
ref_mutD

,riskAllele,Chromosome,MutAllLoc,MapGenLoc,RefGene,MutGene,RefGeneLong
0,T,1,155212373,"155208695,155213824",ACAGAAGTACAATGCTGATTATGATCTGTCAGCTCGGCAAGGGGCA...,ACAGAAGTACAATGCTGATTATGATCTGTCAGCTCGGCAAGGGGCA...,ACCCAAGCCCCAGCCCGGCCTCCGCTCCGGCCGCCGCCACCGCCCC...
1,T,3,27285723,"27106484,27369460",TGAAAATAACACACATGATAGAAATTTCTGTTATGTCTTTCAAAAG...,TGAAAATAACACACATGATAGAAATTTCTGTTATGTCTTTCAAAAG...,TGGAGGGCAACAGTTTTAGCTAAAATTTAATAGCCATTTACAAGCT...
2,A,5,1282299,"1253147,1295068",CTGCACTCCAGCCTGGGCAACAGAGTGAGACTCCATCTCAAAAAAG...,CTGCACTCCAGCCTGGGCAACAGAGTGAGACTCCATCTCAAAAAAG...,ATTAGGATTCAAACATGAGATTTTTTTCAAAACTGAAAAACTCATA...
3,G,7,18935100,"18086949,19002416",CATGGCGACGTTATTTGTAACGACAAAAAAACTGGAATTTGGTGAT...,CATGGCGACGTTATTTGTAACGACAAAAAAACTGGAATTTGGTGAT...,GCAGCGCGCACCGAGCCGGCCGCGCCGCGCCCGCCGCTCTCGCCGC...
4,A,7,79974067,"79768028,80226181",GGATCATATGTTAAGTGCATGTTTAGTTTTGTAAGAAACTGCCAAA...,GGATCATATGTTAAGTGCATGTTTAGTTTTGTAAGAAACTGCCAAA...,GGGAGGCTCGGGCCGCACAGGTGCCCATGGAGGGAGTGAGAGGCTC...
...,...,...,...,...,...,...,...
6396,C,6,53312880,"53267398,53349179",TCCTGTACAATGTTACCACTGGGGGAAACAGGGTAGAGAGTACTTG...,TCCTGTACAATGTTACCACTGGGGGAAACAGGGTAGAGAGTACTTG...,CACAGGTAATCAAAAATGTTTTAATGATTACTGTTTAGACATTTAA...
6397,C,18,53580709,"53568439,53629990",CACCATCTTCTTGTTTTTTTCTTGCAGAAGAAATGTCAAGTTATAC...,CACCATCTTCTTGTTTTTTTCTTGCAGAAGAAATGTCAAGTTATAC...,GTTCTAATGGCTCTCAAGCAAGGTCCACTTGCAGTGAAATAAACCA...
6398,C,5,87315172,"87267883,87391931",GATAAGTGACACCCCCATCTTTAAGTTTGTTTCAGAGATCCTAAAT...,GATAAGTGACACCCCCATCTTTAAGTTTGTTTCAGAGATCCTAAAT...,ACTCACTGAGAGCTCCAGGTAGTGAGCAGTTCAGTCGATTTCCTCG...
6399,C,3,175534948,"174438573,175810548",GGTATTTAAATGATAAAGGAGGCCTATTGACTCAGTAGTTTAGGAC...,GGTATTTAAATGATAAAGGAGGCCTATTGACTCAGTAGTTTAGGAC...,GGATTCAGGTGATGGAGGTGTTTTATGAGCACACACAAAACAAGTG...


In [1]:
X=[2,4,5]
X=X.extend([3,4])
print(X)

None


In [2]:
def Mutate_long(X):
    MutGene=X["RefGene"]
    a=int(X["MapGenLoc"].split(',')[0])
    b=int(X["MutAllLoc"])
    pos_on_gene=b-a
    MutGene=list(MutGene)
    MutGene[pos_on_gene]=X["riskAllele"]
    MutGene=''.join(MutGene)
    return MutGene 
input=[[k,1] for k in ref_mutD["MutGene"]]
input.extend([[k,0] for k in ref_mutD["RefGeneLong"]])
input_long=pd.DataFrame(input_long,columns=["DnaStrand","labels"])
input_raw_long=Dataset.from_pandas(input_long)
"""input_raw_long=input_raw_long.train_test_split(test_size=0.2)
inter_long=input_raw_long["train"].train_test_split(test_size=0.1)
input_raw_long=DatasetDict({
    'train':inter_long["train"],
    'validation':inter_long["test"],
    'test':input_raw_long["test"]
})
print(input_raw_long)
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels=2
model_long = (BertForSequenceClassification.from_pretrained("zhihan1996/DNABERT-2-117M", num_labels=num_labels).to(device))
input_encoded_long = input_raw_long.map(tokenize, batched=True, batch_size=None)
from transformers import Trainer, TrainingArguments
batch_size = 16
logging_steps = len(input_encoded["train"]) // batch_size
model_name_long = "zhihan1996/DNABERT-2-117M-finetuned-cancer-long"
training_args = TrainingArguments(output_dir=model_name_long,
                                num_train_epochs=2,
                                learning_rate=2e-5,
                                per_device_train_batch_size=batch_size,
                                per_device_eval_batch_size=batch_size,
                                weight_decay=0.01,
                                eval_strategy="epoch",
                                disable_tqdm=False,
                                logging_steps=logging_steps,
                                push_to_hub=True,
                                log_level="error")
trainer_long = CustomTrainer(model=model_long, args=training_args,
                compute_metrics=compute_metrics,
                train_dataset=input_encoded_long["train"],
                eval_dataset=input_encoded_long["validation"],
                tokenizer=tokenizer)"""

NameError: name 'ref_mutD' is not defined

In [61]:
trainer.train()

  0%|          | 0/1152 [00:00<?, ?it/s]

{'loss': 0.7038, 'grad_norm': 1.9498193264007568, 'learning_rate': 1e-05, 'epoch': 1.0}


  0%|          | 0/65 [00:00<?, ?it/s]

{'eval_loss': 0.693701982498169, 'eval_accuracy': 0.49073170731707316, 'eval_f1': 0.323086451283361, 'eval_runtime': 16.7472, 'eval_samples_per_second': 61.204, 'eval_steps_per_second': 3.881, 'epoch': 1.0}


KeyboardInterrupt: 